In [3]:
import numpy as np
import pandas as pd
import email
from html.parser import HTMLParser
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from email.parser import BytesParser
from email.policy import default
import csv
import os
import pathlib
import pprint
from os.path import dirname, abspath

In [4]:
d = dirname(dirname(abspath('__file__')))

In [5]:
location = f"{d}/case study 3/"

In [6]:
location

'c:\\Users\\Dylan z4\\OneDrive\\Documents\\GitHub\\qtw\\case study 3/case study 3/'

In [7]:
emails = {"index":[]}
for path, subdirectories, filelist in os.walk(os.path.join(d,"SpamAssassinMessages")):
    for f in filelist:
        
        label = 0 if "ham" in path.lower() else 1
        with open(os.path.join(path, f), "rb") as fp:
            msg = BytesParser(policy=default).parse(fp)
            with open(os.path.join(path, f), "r",encoding= 'utf-8', errors='ignore') as new_f:
                msg = email.message_from_file(new_f)
                for part in msg.walk():
                    if part.get_content_type() == 'text/plain':
                        body = part.get_payload()  # decode
                emails["index"].append({
                    "Subject": msg.get("Subject"),
                    "From": msg.get("from"),
                    "Body": body,
                    "isSpam": label,
                    "FileID": f
                })

df = pd.DataFrame.from_dict(emails["index"])

In [8]:
df["isSpam"].value_counts()

0    6954
1    2400
Name: isSpam, dtype: int64

In [9]:
pd.set_option('display.max_rows', 9000)
pd.set_option('display.max_columns', 200)

Build a spam classifier using naive Bayes and clustering. You will have to create your own dataset from the input messages. Be sure to document how you created your dataset.

In [10]:
df["isSpam"].value_counts()

0    6954
1    2400
Name: isSpam, dtype: int64

In [11]:
df.isna().sum()

Subject    13
From        6
Body        0
isSpam      0
FileID      0
dtype: int64

In [12]:
df = df.dropna(how='any',axis=0) 

In [13]:
df.isna().sum()

Subject    0
From       0
Body       0
isSpam     0
FileID     0
dtype: int64

In [14]:
df["isSpam"].value_counts()

0    6944
1    2397
Name: isSpam, dtype: int64

In [15]:
#remove new lines and such
df = df.replace(r'\\n',' ', regex=True)
df = df.replace(r'\\t',' ', regex=True)
df = df.replace(r'b\'',' ', regex=True)
df = df.replace(r'b\"',' ', regex=True)


In [16]:
df.head()

,Subject,From,Body,isSpam,FileID
1,Re: New Sequences Window,Robert Elz <kre@munnari.OZ.AU>,"Date: Wed, 21 Aug 2002 10:54:46 -05...",0,00001.7c53336b37003a9286aba55d2945844c
2,[zzzzteana] RE: Alexander,Steve Burt <Steve_Burt@cursor-system.com>,"Martin A posted:\nTassos Papadopoulos, the Gre...",0,00002.9c4069e25e1ef370c078db7ee85ff9ac
3,[zzzzteana] Moscow bomber,"""Tim Chapman"" <timc@2ubh.com>",Man Threatens Explosion In Moscow \n\nThursday...,0,00003.860e3c3cee1b42ead714c5c874fe25f7
4,[IRR] Klez: The Virus That Won't Die,Monty Solomon <monty@roscom.com>,Klez: The Virus That Won't Die\n \nAlready the...,0,00004.864220c5b6930b209cc287c361c99af1
5,Re: [zzzzteana] Nothing like mama used to make,Stewart Smith <Stewart.Smith@ee.ed.ac.uk>,"> in adding cream to spaghetti carbonara, whi...",0,00005.bf27cdeaf0b8c4647ecd61b1d09da613


In [17]:
target = df['isSpam']
train = df.drop(['isSpam','FileID'],axis=1)

In [19]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
train['clean_body'] = train['Body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
train = train.drop(['Body'],axis=1)
train.head()

[nltk_data] Downloading package stopwords to C:\Users\Dylan
[nltk_data]     z4\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


,Subject,From,clean_body
1,Re: New Sequences Window,Robert Elz <kre@munnari.OZ.AU>,"Date: Wed, 21 Aug 2002 10:54:46 -0500 From: Ch..."
2,[zzzzteana] RE: Alexander,Steve Burt <Steve_Burt@cursor-system.com>,"Martin A posted: Tassos Papadopoulos, Greek sc..."
3,[zzzzteana] Moscow bomber,"""Tim Chapman"" <timc@2ubh.com>",Man Threatens Explosion In Moscow Thursday Aug...
4,[IRR] Klez: The Virus That Won't Die,Monty Solomon <monty@roscom.com>,Klez: The Virus That Won't Die Already prolifi...
5,Re: [zzzzteana] Nothing like mama used to make,Stewart Smith <Stewart.Smith@ee.ed.ac.uk>,"> adding cream spaghetti carbonara, effect pas..."


In [20]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.35, shuffle=True, random_state=45)

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cv_data_train = cv.fit_transform(X_train)
cv_data_test = cv.transform(X_test)